An example of fetching archived data using the ArchiverEventSource interface

In [1]:
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import h5py

import pandas as pd
import pprint

import pymongo
from pymongo import MongoClient

import requests

import databroker
from databroker import list_configs
from databroker import Broker

# make graphics inline
%matplotlib inline

from archiver import ArchiverEventSource

## Databroker Header

In [2]:
db = Broker.named('hxn_new')
header = db[-2]
header['start']['uid']

'dff0dab0-08ed-478d-b041-d38891430b24'

## Archiver Event Source

In [3]:
url = 'http://xf07bm-ca1.cs.nsls2.local:17668/'
timezone = 'US/Eastern'
pvs = {"PV1" : "XF:07BMA-VA{Diag:1-IP:1}I-I", 
       "PV2" : "FE:C07B-OP{Mir:Col-TC:4}T-I"}
es = ArchiverEventSource(url, timezone, pvs)

### stream_names_given_header

In [4]:
stream_names = es.stream_names_given_header(header)
print(stream_names)

['archiver_PV1', 'archiver_PV2']


### fields_given_header

In [5]:
fset = es.fields_given_header(header)
fields = list(fset)
print(len(fields), fields[0], fields[1])

2 PV1 PV2


### descriptors_given_header

In [6]:
descriptors = es.descriptors_given_header(header)
len(descriptors)

2

In [7]:
print(descriptors[0])

{'external_query': {'to': '2018-04-20T02:33:57-04:00', 'from': '2018-04-20T02:13:19-04:00', 'pv': 'XF:07BMA-VA{Diag:1-IP:1}I-I'}, 'run_start': 'dff0dab0-08ed-478d-b041-d38891430b24', 'external_url': 'http://xf07bm-ca1.cs.nsls2.local:17668/', 'time': 1524204799.9761238, 'data_keys': {'PV1': {'dtype': 'number', 'shape': [], 'source': 'XF:07BMA-VA{Diag:1-IP:1}I-I'}}, 'uid': 'empheral-4f154eff-2c8f-4e8c-a511-fe87053dd06c'}


In [8]:
print(descriptors[1])

{'external_query': {'to': '2018-04-20T02:33:57-04:00', 'from': '2018-04-20T02:13:19-04:00', 'pv': 'FE:C07B-OP{Mir:Col-TC:4}T-I'}, 'run_start': 'dff0dab0-08ed-478d-b041-d38891430b24', 'external_url': 'http://xf07bm-ca1.cs.nsls2.local:17668/', 'time': 1524204799.9761238, 'data_keys': {'PV2': {'dtype': 'number', 'shape': [], 'source': 'FE:C07B-OP{Mir:Col-TC:4}T-I'}}, 'uid': 'empheral-384e0873-e708-4489-b35d-4549cf3966c2'}


### docs_given_header

In [9]:
gen_es = es.docs_given_header(header)
names_es = []
docs_es = []
for name, doc in gen_es:
    names_es.append(name)
    docs_es.append(doc)
print(len(names_es), len(docs_es))

736 736


In [10]:
# descriptor
docs_es[1]

{'data_keys': {'PV1': {'dtype': 'number',
   'shape': [],
   'source': 'XF:07BMA-VA{Diag:1-IP:1}I-I'}},
 'external_query': {'from': '2018-04-20T02:13:19-04:00',
  'pv': 'XF:07BMA-VA{Diag:1-IP:1}I-I',
  'to': '2018-04-20T02:33:57-04:00'},
 'external_url': 'http://xf07bm-ca1.cs.nsls2.local:17668/',
 'run_start': 'dff0dab0-08ed-478d-b041-d38891430b24',
 'time': 1524204799.9761238,
 'uid': 'empheral-4f154eff-2c8f-4e8c-a511-fe87053dd06c'}

In [11]:
# event
docs_es[3]

{'data': {'PV1': 1.2e-06},
 'descriptor': 'empheral-4f154eff-2c8f-4e8c-a511-fe87053dd06c',
 'seq_num': 1,
 'time': 1524204800,
 'timestamps': {'PV1': 1524204800},
 'uid': 'ephemeral-7892bdc8-5cf4-496a-b0e2-726a6fd15101'}

### table_given_header

In [12]:
df = es.table_given_header(header)

In [13]:
df

,time,PV1,PV2
1,2018-04-20 06:13:17,1.1e-06,
2,2018-04-20 06:13:18,,29.4
3,2018-04-20 06:13:19,,29.5
4,2018-04-20 06:13:20,1.2e-06,29.4
5,2018-04-20 06:13:21,1.1e-06,
6,2018-04-20 06:13:24,,29.5
7,2018-04-20 06:13:25,,29.4
8,2018-04-20 06:13:26,1.2e-06,
9,2018-04-20 06:13:27,1.1e-06,
10,2018-04-20 06:13:28,,29.3
